In [1]:
from mio import *
set_mio_log_level('debug')


time="2024-07-16T09:09:44+02:00" level=info msg="core.syncTime[time.go:34] - clock offset 805.27µs from time.google.com "


# Introduction
Mio is a library designed to store and share protected information. It allows information to be stored in public and cloud locations (e.g., S3) while ensuring that only intended peers have access, and not the storage service providers.

### Identity
Each peer in the information sharing is identified by a cryptographic key, which allows for identification via signature and asymmetric encryption of data. An identity also includes a nickname, a human-readable name for easier management.

An identity consists of a public ID, which is a combination of the nickname and the public part of the cryptographic key.

In [2]:
i = Identity('Admin')  # New identity with name

print('ID: ', i.id)
print('Nick: ', i.nick)
print('Private key', i.private)

ID:  Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!
Nick:  Admin
Private key Ebe9rdMxsQXhDZkJLJzm4iTd9+gAE89iDAy4P4IE0MkL_Q87C1Bpeaj3Awzg3XmCazEXPGt1DaKKPUOerwGhCXX96j_GKr+hcxluCk3NfFnBD5wqcmzkcxMPkzhyEIZC


### Local DB
The library requires a local SQLite database to manage metadata and encryption keys. By default, this database is named _mio.db_ and is created in the user's configuration folder.

In [3]:
DB.default()

time="2024-07-16T09:09:46+02:00" level=info msg="sqlx.(*DB).Define[define.go:38] - SQL Init stmt (line 1) 'CREATE TABLE IF NOT EXISTS mio_configs (\nnode    VARCHAR(128) NOT NULL,\nk       VARCHAR(64) NOT NULL,\ns       VARCHAR(64) NOT NULL,\ni       INTEGER NOT NULL,\nb       BLOB,\nCONSTRAINT pk_safe_key PRIMARY KEY(node,k)\n);\n' executed\n"
time="2024-07-16T09:09:46+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_GET_CONFIG' (11) 'SELECT s, i, b FROM mio_configs WHERE node=:node AND k=:key\n'\n"
time="2024-07-16T09:09:46+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_SET_CONFIG' (14) 'INSERT INTO mio_configs(node,k,s,i,b) VALUES(:node,:key,:s,:i,:b)\n\tON CONFLICT(node,k) DO UPDATE SET s=:s,i=:i,b=:b\n\tWHERE node=:node AND k=:key\n'\n"
time="2024-07-16T09:09:46+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_DEL_CONFIG' (19) 'DELETE FROM 

/home/ea/.config/mio.db

### Safe
The information container is called a safe. The safe integrates a local database, the identity, and a storage URL to create a protected information storage system.

As of this writing, the library supports the following storage media: local filesystem, SFTP, S3, and WebDav. The table below shows the URL format for each storage type. The URL path includes the public ID of the safe's creator and a user-friendly name for the safe.

|Media|Format|Sample|
|-|-|-|
|local|file://_path_/_id_/_name_|file:///tmp/mio/Admin.A3Z1CR0wYMK_gXsRLkpowC3dVFC5rUNEeakiWPyb3D5l5VD1SXEFTxzQKEIvzNvKGEZGYp4yETo77SN+ViGP_00!/sample|
|S3|s3://_server_/_bucket_/_id_/_name_?a=_access_&s=_secret_|s3://d551285d92ed8fa4048fc09ca9113568.r2.cloudflarestorage.com/mio/Admin.A3Z1CR0wYMK_gXsRLkpowC3dVFC5rUNEeakiWPyb3D5l5VD1SXEFTxzQKEIvzNvKGEZGYp4yETo77SN+ViGP_00!/sample?a=acc5aba2f85d63536bbb45f085bb2b23&s=bcc49533aaaa46d929282b542ce598a12f7a4522dac8d5e9d403b39629484c2b&region=auto|


A new safe is created using the _create_ method.

In [4]:
url = 'file:///tmp/mio/{}/sample'.format(i)
s = Safe.create(DB.default(), i, url)
s

time="2024-07-16T09:09:47+02:00" level=info msg="sqlx.(*DB).Define[define.go:38] - SQL Init stmt (line 1) 'CREATE TABLE IF NOT EXISTS mio_configs (\nnode    VARCHAR(128) NOT NULL,\nk       VARCHAR(64) NOT NULL,\ns       VARCHAR(64) NOT NULL,\ni       INTEGER NOT NULL,\nb       BLOB,\nCONSTRAINT pk_safe_key PRIMARY KEY(node,k)\n);\n' executed\n"
time="2024-07-16T09:09:47+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_GET_CONFIG' (11) 'SELECT s, i, b FROM mio_configs WHERE node=:node AND k=:key\n'\n"
time="2024-07-16T09:09:47+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_SET_CONFIG' (14) 'INSERT INTO mio_configs(node,k,s,i,b) VALUES(:node,:key,:s,:i,:b)\n\tON CONFLICT(node,k) DO UPDATE SET s=:s,i=:i,b=:b\n\tWHERE node=:node AND k=:key\n'\n"
time="2024-07-16T09:09:47+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_DEL_CONFIG' (19) 'DELETE FROM 

file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample

At creation, only the creator belongs to the safe in both the _adm_ and _usr_ groups. Similar to Unix operating systems, a safe has multiple groups that define the permissions for different users (identities). The _get_groups_ method displays the available groups and their associated users.

In [5]:
s.get_groups()

time="2024-07-16T09:09:49+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"


{'adm': {'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!': True},
 'usr': {'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!': True}}

A user can be added to the safe using the _update_group_ method

In [6]:
alice = Identity('Alice')
s.update_group('usr', Safe.grant, [alice.id])

time="2024-07-16T09:09:50+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T09:09:50+02:00" level=info msg="safe.applyChange[group.go:338] - group change applied: Alice granted to usr by Admin"
time="2024-07-16T09:09:50+02:00" level=info msg="safe.(*Safe).UpdateGroup[group.go:133] - group change created and added to the chain: Alice granted to usr by Admin"
time="2024-07-16T09:09:50+02:00" level=info msg="safe.writeGroupChanges[group.go:447] - group changes written to the store on batches [0]"
time="2024-07-16T09:09:51+02:00" level=info msg="safe.writeKeystore[key.go:237] - keystore usr.ks written successfully by Admin: users Alice, Admin"


{'adm': {'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!': True},
 'usr': {'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!': True,
  'Alice.A0Me7TDwg0pBl8CbxEKLuwC3_WVP9Rqu1U1nkWCRF7CLQUzMxOGfczncxrMQtT5+a1LicmxNiDoRGg4Q3nelqIU!': True}}

To open an existing safe, use the _open_ method.

In [23]:
url = 'file:///tmp/mio/{}/sample'.format(i)
s = Safe.open(DB.default(), i, url)
s

time="2024-07-16T17:33:45+02:00" level=info msg="sqlx.(*DB).Define[define.go:38] - SQL Init stmt (line 1) 'CREATE TABLE IF NOT EXISTS mio_configs (\nnode    VARCHAR(128) NOT NULL,\nk       VARCHAR(64) NOT NULL,\ns       VARCHAR(64) NOT NULL,\ni       INTEGER NOT NULL,\nb       BLOB,\nCONSTRAINT pk_safe_key PRIMARY KEY(node,k)\n);\n' executed\n"
time="2024-07-16T17:33:45+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_GET_CONFIG' (11) 'SELECT s, i, b FROM mio_configs WHERE node=:node AND k=:key\n'\n"
time="2024-07-16T17:33:45+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_SET_CONFIG' (14) 'INSERT INTO mio_configs(node,k,s,i,b) VALUES(:node,:key,:s,:i,:b)\n\tON CONFLICT(node,k) DO UPDATE SET s=:s,i=:i,b=:b\n\tWHERE node=:node AND k=:key\n'\n"
time="2024-07-16T17:33:45+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'MIO_DEL_CONFIG' (19) 'DELETE FROM 

file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample

### Filesystem
A filesystem provides a file-oriented interface on top of a safe. Use the _put_data_ method to write a new file and the _list_ method to display the updated directory contents.

In [7]:
fs = s.fs()
fs.put_data('sub/test.txt', b'test')
fs.list()

time="2024-07-16T09:09:51+02:00" level=info msg="fs.(*FileSystem).putSync[put.go:122] - putting file 2d3efe172472000 from data"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.writeBody[put.go:195] - writing body to fs/data/2d3efe172472000"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.writeHeader[file.go:83] - encrypting header sub/test.txt"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.writeHeader[file.go:89] - writing header sub/test.txt to fs/headers/82719d195d0fc2f5/2d3efe172472000"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.syncHeaders[file.go:140] - found 1 headers in sub"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.readHeader[file.go:131] - read header sub/test.txt with id 203770159725748224"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.writeFileToDB[file.go:203] - stored dir sub"
time="2024-07-16T09:09:51+02:00" level=info msg="fs.writeFileToDB[file.go:207] - stored file sub/test.txt with id 203770159725748224, args map[attributes:map[] 

[{'id': 0,
  'dir': '',
  'name': 'sub',
  'isDir': True,
  'groupName': '',
  'creator': '',
  'size': 0,
  'modTime': '1970-01-01T01:00:00+01:00',
  'tags': [''],
  'attributes': None,
  'localCopy': '',
  'copyTime': '1970-01-01T01:00:00+01:00',
  'encryptionKey': ''}]

In [8]:
fs.list('sub')

time="2024-07-16T09:09:56+02:00" level=info msg="fs.searchFiles[file.go:243] - found file sub/test.txt"
time="2024-07-16T09:09:56+02:00" level=info msg="fs.searchFiles[file.go:245] - found 1 files in sub with search options map[#orderBy: after:-6795364578871345152 before:-6795364578871345152 creator: dir:sub groupName: limit:100 name: offset:0 prefix: safeID:file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample suffix: tag:]"


[{'id': 203770159725748224,
  'dir': 'sub',
  'name': 'test.txt',
  'isDir': False,
  'groupName': 'usr',
  'creator': 'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!',
  'size': 4,
  'modTime': '2024-07-16T09:09:51.946423794+02:00',
  'tags': [''],
  'attributes': None,
  'localCopy': '',
  'copyTime': '2024-07-16T09:09:51.947328251+02:00',
  'encryptionKey': 'DXzL50n5RJTr7SozEBT/UlLhpqXR0YKcEGyA3gqwnDIhRbL9tOfG0ew2Vwew1iT7'}]

In [9]:
fs.get_data('sub/test.txt')

b'test'

In [10]:
fs.delete('sub/test.txt')
fs.list()

time="2024-07-16T17:26:30+02:00" level=info msg="fs.(*FileSystem).List[list.go:23] - syncing headers of "
time="2024-07-16T17:26:30+02:00" level=info msg="fs.searchFiles[file.go:245] - found 0 files in  with search options map[#orderBy: after:-6795364578871345152 before:-6795364578871345152 creator: dir: groupName: limit:100 name: offset:0 prefix: safeID:file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample suffix: tag:]"


[]

### Distributed Database
This is an SQLite API that propagates changes to other peers through a secure mechanism.

When creating a new instance, it must be associated with a group used for data transfer through this secure channel. Only users within the specified group will receive updates. During instance creation, you can provide initialization DDL statements; each initialization SQL statement must be preceded by the comment -- INIT to ensure proper processing and setup of the database.


In [11]:
ddl = """
-- INIT
CREATE TABLE IF NOT EXISTS animal (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT)
"""

ddls = {1.0: ddl}
d = s.database('usr', ddls)

time="2024-07-16T17:26:33+02:00" level=info msg="sqlx.(*DB).Define[define.go:38] - SQL Init stmt (line 2) 'CREATE TABLE IF NOT EXISTS animal (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT)\n' executed\n"
time="2024-07-16T17:26:33+02:00" level=info msg="db.Open[db.go:32] - Opening database on safe file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample with group usr"


The below code insert a row in the table and then retrieve all the values

In [12]:
d.exec('INSERT INTO animal (name) VALUES (:name)', name = 'cat')
rows = d.query("SELECT * FROM animal ")
for row in rows:
    print(row)

[1, 'cat']
[2, 'cat']
[3, 'cat']
[4, 'cat']
[5, 'cat']
[6, 'cat']


The methods _exec_ and _select_ operate locally. Only when the method _sync_ is called, changes propagates to other peers and at the same outgoing changes are applied.

In [13]:
d.sync()

time="2024-07-16T17:26:37+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"


#### Placeholders

The API includes a mechanism for defining placeholders within initialization SQL, which enhances both readability and performance. To create a placeholder, add a comment prefix (--) followed by the placeholder name, and then write the SQL query you want to assign to that placeholder. This placeholder can then be used in place of the actual query in exec and query operations. Using placeholders not only makes your SQL more readable but also allows the API to cache the statement, improving execution efficiency.

In [14]:
ddl = """
-- INIT
CREATE TABLE IF NOT EXISTS animal (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT)

-- INSERT_ANIMAL
INSERT INTO animal (name) VALUES (:name)

-- SELECT_ANIMALS
SELECT * FROM animal

-- PRUNE_ANIMALS
DELETE FROM animal WHERE id NOT IN (
    SELECT id FROM animal
    ORDER BY id
    LIMIT 5
);

"""

ddls = {1.0: ddl}
d = s.database('usr', ddls)

d.exec('INSERT_ANIMAL', name = 'cat')
rows = d.query('SELECT_ANIMALS')
for row in rows:
    print(row)


[1, 'cat']
[2, 'cat']
[3, 'cat']
[4, 'cat']
[5, 'cat']
[6, 'cat']
[7, 'cat']


time="2024-07-16T17:26:39+02:00" level=info msg="sqlx.(*DB).Define[define.go:38] - SQL Init stmt (line 2) 'CREATE TABLE IF NOT EXISTS animal (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT)\n' executed\n"
time="2024-07-16T17:26:39+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'INSERT_ANIMAL' (5) 'INSERT INTO animal (name) VALUES (:name)\n'\n"
time="2024-07-16T17:26:39+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'SELECT_ANIMALS' (8) 'SELECT * FROM animal\n'\n"
time="2024-07-16T17:26:39+02:00" level=info msg="sqlx.(*DB).prepareStatement[define.go:81] - SQL statement compiled: 'PRUNE_ANIMALS' (11) 'DELETE FROM animal WHERE id NOT IN (\nSELECT id FROM animal\nORDER BY id\nLIMIT 5\n);\n'\n"
time="2024-07-16T17:26:39+02:00" level=info msg="db.Open[db.go:32] - Opening database on safe file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample 

In [15]:
d.sync()
d.exec('PRUNE_ANIMALS')

time="2024-07-16T17:26:41+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"


2

### Communication
A communication is a messaging interface for peer-to-peer communication or  broadcast to all users in a group. 
For instance a broadcast to all users in group _usr_ only requires two lines. A message can contain _text_, binary _data_ or the location of a _file_ to add as attachment.

In [16]:
c = s.comm() #Comm is a service on top of a safe to 
c.broadcast('usr', text='hello')

{'Text': 'hello', 'Data': '', 'File': ''}


time="2024-07-16T17:26:42+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:42+02:00" level=info msg="comm.(*Comm).send[send.go:79] - message for id 203895192557133824 saved to comm/usr/2d46198ebc72000"


Receiving messages is straightforward as well. By default, the _receive_ function returns messages for the current user and the groups to which the user belongs. Optionally the parameter _filter_ can restrict the retrieval to a specific group or user.

In [17]:
c.receive()

time="2024-07-16T17:26:42+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:42+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:42+02:00" level=info msg="comm.(*Comm).Receive[receive.go:64] - received 1 messages from safe file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample"


[{'sender': 'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!',
  'encryptionId': 0,
  'recipient': 'usr',
  'id': 203895192557133824,
  'text': 'hello',
  'data': '',
  'file': ''}]

Receiving a message consumes it, ensuring that previously examined messages are not returned in subsequent calls.

In [18]:
ms = c.receive()
print('new messages:', ms)

new messages: []


time="2024-07-16T17:26:43+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:43+02:00" level=info msg="comm.(*Comm).Receive[receive.go:64] - received 0 messages from safe file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample"


It is still possible to receive older messages by rewinding to a specific message ID or to 0 to start from the first message.

In [19]:
c.rewind('usr', 0)
c.receive()

time="2024-07-16T17:26:44+02:00" level=info msg="comm.(*Comm).Rewind[comm.go:29] - rewinded communication for usr to id 0"
time="2024-07-16T17:26:44+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:44+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:44+02:00" level=info msg="comm.(*Comm).Receive[receive.go:64] - received 1 messages from safe file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample"


[{'sender': 'Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!',
  'encryptionId': 0,
  'recipient': 'usr',
  'id': 203895192557133824,
  'text': 'hello',
  'data': '',
  'file': ''}]

#### Attachments
A message can include an attached file by specifying its location in the local file system.

In [20]:
data = os.urandom(1 * 1024 * 1024)
with open('content.in', 'wb') as f:
    f.write(data)

c.send(i.id, text='file to myself', file='content.in')
os.remove('content.in')

time="2024-07-16T17:26:46+02:00" level=info msg="comm.(*Comm).send[send.go:52] - message file for id 203895209950912512 saved to comm/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/2d4619cf8872000.data"
time="2024-07-16T17:26:46+02:00" level=info msg="comm.(*Comm).send[send.go:79] - message for id 203895209950912512 saved to comm/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/2d4619cf8872000"


In [21]:
m = c.receive()[0]
m['file']

time="2024-07-16T17:26:49+02:00" level=info msg="safe.SyncGroupChain[group.go:187] - group chain is up to date, using the local copy"
time="2024-07-16T17:26:49+02:00" level=info msg="comm.(*Comm).Receive[receive.go:64] - received 1 messages from safe file:///tmp/mio/Admin.ArsbgIrlX148mKQ9sNZ7Gu7Xpo2hCU8oBztkVf_0_89rdf3qP8Yqv6FzGW4KTc18WcEPnCpybORzEw+TOHIQhkI!/sample"


'content.in'

In [22]:
c.download(m, 'content.out')
with open('content.out', 'rb') as file:
    content = file.read()

if content == data:
    print('Cool! the same')
os.remove('content.out')

Cool! the same
